## Cumulative sums

csc596: Accelerate! GPU
<br>
university of miami
<br>
fall 2022-2023 (231)
<br>
Burton Rosenberg

<br>last update:
- Sep 28, 2022

In [ ]:

# the partial sum problem: 
#  given array a[i], calculate for each j, sum_{i<j} a[i]

# solution in place, sequential, linear time:

def partial_sum_seq(a):
    for i in range(1,len(a)):
        a[i] += a[i-1]
    return

# parallel for a GPU. Method 1

# in the i-th phase the blocks are of size B=2^i, from 1 increasing
#   until n/2

# notation: indexing by block and offset in block can be a[(j,i)] 
#   where j=0,...,n/B-1, and i=0,...,B-1
#   and a[k] = a[(j,i)] by k = i + B*j, or (j,i) = (k//B,k%B)

# Loop Invariant: each block individually achieves the partial sum over that block.
#   let a'[k] be the original values of a[k], then 
#   for all (b,i), a[(b,i)] = sum_{j<=i} a'[(b,j)]

# Loop update: double the block size to B'=2*B and reassert the invariant.
#   define the ladder of an index k by the parity of the i-th bit. 
#     if odd then k it is the top rung, if even it is the lower rung
#   for all top rung b, for all i in [0,B),  a'[(b,i)] += a'[(b-1,B-1)]

#   NOTE: this can be done in parallel, as all writes are independent
#   the read is a broadcast read of the elements a'[(b-1,2^i-1)] for all b
#   this is n/2^i reads and n/2 writes in phase i

# There are log(n) phases
#

def partial_sum_ladder(a):
    phase = 1
    phase_mask = ~(phase-1)
    while phase<len(a):
        print(phase)
        
        for thread in range(len(a)):
            if phase & thread != 0:
                prev_rung = (thread&phase_mask)-1
                print(f"\t{thread}, {prev_rung}")
                a[thread] += a[prev_rung]
                
            else:
                # thread sleeps
                None

        phase <<= 1
        phase_mask = ~(phase-1)
        
    return


# the folding method has blocks of size 2^h, supposing there are 2^j 
# such blocks, stack them one above the other such that the 0 cell is
# the uppermost, leftmost cell. The Loop Invariant is that the i-th 
# entry is the sum of all elements that would have been above it in 
# the original array values. 
# Update is to divide each block in half, fold the right half under the
# left half, and add to a cell the value of the corresponding cell 
# just above the cell.
# This update is down in parallel, such that all threads read the current
# cell values and updates simultaneously to the new cell values. 
# As a sequential program this can be done by working from highest 
# indexed blocks to lowest indexed blocks


def partial_sum_folded(a):
    m = len(a)
    n = 1
    while n<m:
        n *= 2
    phase = n//2
    while True:
        print(f'fphase {phase}')
        for i in range(m):
            if (i+phase) < m :
                a[i] += a[i+phase]
        print(a)
        if phase==1:
            break
        phase = phase//2
    return a



In [ ]:
# test procedures


#--------


SIZE = 30
a = [1]*SIZE
#fill_array(a)
print(partial_sum_seq(a))

a = [2]*SIZE
partial_sum_ladder(a)
print(a)

In [ ]:
SIZE = 33
a = [1]*SIZE
#fill_array(a)
print(partial_sum_folded(a))